In [2]:
### TOP5 PAPERS ONLY TITLE AND ABSTRACT ####

#Citation score import requests
from concurrent.futures import ThreadPoolExecutor
from urllib.error import HTTPError
from Bio import Entrez
import pandas as pd
import requests
import time
import os

# Get the current working directory
current_directory = os.getcwd()

# Print the current working directory
print(f"My current working directory is: {current_directory}")
os.chdir('/Users/kiliankleemann/Dropbox/                                       $$$NRN/5-Figures_Boxes_Tables/Tables/Table-2 GWAS risk factors enriched in immune cells (Rudy_Genome Medicine)/Rudy_GWAS_analysis/Immune_cells')      


My current working directory is: /Users/kiliankleemann/Dropbox/                                       $$$NRN/5-Figures_Boxes_Tables/Tables/Table-2 GWAS risk factors enriched in immune cells (Rudy_Genome Medicine)/Rudy_GWAS_analysis/Immune_cells
Top 5 papers for T cell:
 Empty DataFrame
Columns: [gene]
Index: [] 

Top 5 papers for B cell:
 Empty DataFrame
Columns: [gene]
Index: [] 

Top 5 papers for neutrophil:
 Empty DataFrame
Columns: [gene]
Index: [] 

Top 5 papers for monocyte:
 Empty DataFrame
Columns: [gene]
Index: [] 

Top 5 papers for macrophage:
 Empty DataFrame
Columns: [gene]
Index: [] 

Top 5 papers for NK cell:
 Empty DataFrame
Columns: [gene]
Index: [] 



/var/folders/p1/w5vpt8z97n3f8xvc8kzc87nc0000gn/T/ipykernel_22313/748560432.py:107: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  top_5_per_gene = df.groupby('gene').apply(lambda x: x.nlargest(5, 'citation_count')).reset_index(drop=True)
/var/folders/p1/w5vpt8z97n3f8xvc8kzc87nc0000gn/T/ipykernel_22313/748560432.py:107: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior 

In [ ]:
# Function to search EuropePMC for a given gene and cell type and return the top 5 research papers TITLE OR ABSTRACT
def get_top_papers_europepmc(gene, celltype):
    # Introduce functional relationship keywords
    functional_keywords = "(expression OR regulation OR activation OR function OR signaling OR interaction)"
    
    # Construct the query and exclude reviews
    query = f'(TITLE:"{gene}" OR ABSTRACT:"{gene}") AND (TITLE:"{celltype}" OR ABSTRACT:"{celltype}") AND {functional_keywords} AND PUB_TYPE:research-article AND (((SRC:MED OR SRC:PMC OR SRC:AGR OR SRC:CBA) NOT (PUB_TYPE:"Review")))'
    
    url = 'https://www.ebi.ac.uk/europepmc/webservices/rest/search'
    params = {
        'query': query,
        'format': 'json',
        'resultType': 'core',
        'pageSize': 5000  # Fetch up to 100 results per request
    }

    try:
        response = requests.get(url, params=params)
        response.raise_for_status()
        data = response.json()

        # Extract relevant information and sort by citation count
        papers = [
            {
                'title': result.get('title', 'No title available'),
                'author': result.get('authorString', 'No authors available'),
                'journal': result.get('journalTitle', 'No journal available'),
                'year': result.get('pubYear', 'No year available'),
                'citation_count': int(result.get('citedByCount', 0)),
                'doi': result.get('doi', 'No DOI available')
            }
            for result in data.get('resultList', {}).get('result', [])
        ]
        
        # Sort papers by citation count in descending order and get the top 5
        top_papers = sorted(papers, key=lambda x: x['citation_count'], reverse=True)[:5]
        
        return top_papers
    except requests.exceptions.HTTPError as err:
        print(f"HTTP error occurred: {err}")
    except Exception as err:
        print(f"An error occurred: {err}")

    return []

# Import gene list from an Excel sheet
excel_file = "GWAS_genes.xlsx"  # Replace with your file path
sheet_name = "GWAS_genes"  # Replace with the correct sheet name if needed

# Assuming the Excel file has a column named "Gene" with your gene list
genes_df = pd.read_excel(excel_file, sheet_name=sheet_name)
genes = genes_df['Gene'].tolist()

# Define your list of cell types
celltypes = ["T cell", "B cell", "neutrophil", "monocyte","macrophage", "NK cell", "endothelial"]  # Replace with your list of cell types

# Create an empty dictionary to store DataFrames for each cell type
celltype_paper_dfs = {}

# Function to process a single gene and all cell types
def process_gene(gene):
    gene_papers = {}
    for celltype in celltypes:
        top_papers = get_top_papers_europepmc(gene, celltype)
        gene_papers[celltype] = top_papers
        time.sleep(1)  # Adding a 1-second delay to avoid overwhelming the API
    return gene_papers

# Process each gene and store results in the dictionary
for gene in genes:
    gene_papers = process_gene(gene)
    for celltype in celltypes:
        # Convert the list of papers to a DataFrame
        df = pd.DataFrame(gene_papers[celltype])
        
        # Append the gene information to each DataFrame
        df['gene'] = gene
        
        # Store the DataFrame in the dictionary, creating a new one for each cell type
        if celltype in celltype_paper_dfs:
            celltype_paper_dfs[celltype] = pd.concat([celltype_paper_dfs[celltype], df], ignore_index=True)
        else:
            celltype_paper_dfs[celltype] = df

# Save each cell type's DataFrame to a separate Excel file, only keeping the top 5 papers
for celltype, df in celltype_paper_dfs.items():
    # Group by gene and take only the top 5 papers for each gene
    top_5_per_gene = df.groupby('gene').apply(lambda x: x.nlargest(5, 'citation_count')).reset_index(drop=True)
    
    output_file = f"top_5_papers_all_GWAS_{celltype.replace(' ', '_')}.xlsx"
    top_5_per_gene.to_excel(output_file, index=False)

# Print the DataFrame for each cell type (optional)
for celltype, df in celltype_paper_dfs.items():
    print(f"Top 5 papers for {celltype}:\n", df.head(), "\n")